In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from sklearn.metrics import *
from copy import deepcopy
import matplotlib.pyplot as plt
import math


from tqdm.auto import tqdm
import seaborn as sns

from src.helpers import *
from src.visualize import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
from torch.utils.data import Dataset
import pickle

import optuna
from src.optuna_functions import * 


device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32

import sys
sys.path.append("c:\\Users\\nerea\\OneDrive\\Documentos\\EPFL_MASTER\\PDM\\Project\\PyalData")
# to change for the actual path where PyalData has been cloned

c:\Users\nerea\anaconda3\envs\sinthlab\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_to_data = './Data/Processed_Data'
baseline_data = os.path.join(path_to_data, 'Tidy_Sansa_11_04.pkl')

with open(baseline_data, 'rb') as fp:
        df_baseline = pickle.load(fp)


X_train, y_train, X_val, y_val, X_test,\
    y_test, info_train, info_val, info_test = \
    train_test_split(df_baseline, train_variable = 'both_rates', \
                     target_variable = 'target_pos', num_folds = 5)

# Test one of the folds first
fold_num = 'fold0'
fold = 0

print('We are testing the optimization method on fold ', fold)


X_train = X_train[fold_num]
X_val = X_val[fold_num]
X_test = X_test[fold_num]
y_test = y_test[fold_num]
y_train = y_train[fold_num]
y_val = y_val[fold_num]


# Specify that we want our tensors on the GPU and in float32
device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32
path_to_models = './Models'

num_dim_output = y_train.shape[1]
num_features = X_train.shape[1]

seq_length = 75

# Reshape x_train to match the number of columns in the model's input layer
xx_train = X_train.reshape(X_train.shape[0] // seq_length, seq_length, X_train.shape[1])  
# Reshape y_train to match the number of neurons in the model's output layer
yy_train = y_train.reshape(y_train.shape[0] // seq_length, seq_length, y_train.shape[1])  

xx_val = X_val.reshape(X_val.shape[0] // seq_length, seq_length, X_val.shape[1])  
yy_val = y_val.reshape(y_val.shape[0] // seq_length, seq_length, y_val.shape[1])  

xx_test = X_test.reshape(X_test.shape[0] // seq_length, seq_length, X_test.shape[1])  
yy_test = y_test.reshape(y_test.shape[0] // seq_length, seq_length, y_test.shape[1])  

seed = 42
torch.manual_seed(seed)

Reg = globals().get(Regularizer_LSTM)


Test trials  11
Val trials 9
We are testing the optimization method on fold  0


In [3]:
# Fit the LSTM model
def train_model_optuna(trial):
    
    X = xx_train
    Y = yy_train
    X_val = xx_val
    Y_val = yy_val

    num_epochs= 1000
    early_stop = 5

    model = CausalTemporalLSTM_Optuna(trial, num_features= num_features, 
                out_dims = num_dim_output).to(device)
    
    # Set up the optimizer with the specified learning rate
    #optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer_name = 'Adam'
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log = True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr = lr)

    lr_step_size = 10 # trial.suggest_int('lr_step_size', 5, 15)
    lr_gamma = trial.suggest_float('lr_gamma', 1.05, 1.4)
    # Set up a learning rate scheduler
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=lr_step_size, 
                                    gamma=lr_gamma)
        
    # Keep track of the best model's parameters and loss
    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e8

    # Enable anomaly detection for debugging
    torch.autograd.set_detect_anomaly(True)

    # Track the train and validation loss
    train_losses = []
    val_losses = []
    # Counters for early stopping
    not_increased = 0
    end_train = 0
    
    # Reshape data for the LSTM
    seq_length_LSTM = trial.suggest_int('seq_length_LSTM', 5, 20)
    train_dataset = SequenceDataset(Y,X,seq_length_LSTM)
    val_dataset = SequenceDataset(Y_val,X_val,seq_length_LSTM)


    batch_size_train = trial.suggest_int('batch_size_train', 25, 50)
    batch_size_val = trial.suggest_int('batch_size_val',25, 50)
    loader_train = data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    loader_val = data.DataLoader(val_dataset, batch_size=batch_size_val, shuffle=True)

    # hyperparameter for huber loss
    delta = 8 #trial.suggest_int('huber_delta', 5, 10)

    # hyperparameter for regularizer
    alpha = 1e-5 #trial.suggest_float('alpha_reg', 1e-7, 1e-3, log = True)
    l1_ratio = 0.5 #trial.suggest_float('l1_ratio_reg', 0.4, 0.6)

    # Loop through epochs
    for epoch in np.arange(num_epochs):
        for phase in ['train', 'val']:
            # set model to train/validation as appropriate
            if phase == 'train':
                model.train()
                loader = loader_train
            else:
                model.eval()
                loader = loader_val

            # Initialize variables to track loss and batch size
            running_loss = 0
            running_size = 0        

            # Iterate over batches in the loader
            for X_, y_ in loader:
                #X_ = X_.to('cuda')
                #y_ = y_.to('cuda')
                if phase == "train":
                    with torch.set_grad_enabled(True):
                        optimizer.zero_grad()

                        output_t = model(X_)
                        output_t = torch.squeeze(output_t)


                        loss_t = huber_loss(output_t, y_, delta)
                        
                        # Add regularization to the loss in the training phase
                        loss_t += Regularizer_LSTM(model, l1_ratio, alpha)

                        # Compute gradients and perform an optimization step
                        loss_t.backward()
                        optimizer.step()
                else:
                    # just compute the loss in validation phase
                    output_t = model(X_)
                    output_t = torch.squeeze(output_t)

                    loss_t = huber_loss(output_t, y_, delta)

                # Ensure the loss is finite
                assert torch.isfinite(loss_t)
                running_loss += loss_t.item()
                running_size += 1

            # compute the train/validation loss and update the best
            # model parameters if this is the lowest validation loss yet
            running_loss /= running_size
            if phase == "train":
                train_losses.append(running_loss)
            else:
                val_losses.append(running_loss)
                
                # Update best model parameters if validation loss improves
                if running_loss < best_loss:
                    best_loss = running_loss
                    best_model_wts = deepcopy(model.state_dict())
                    not_increased = 0
                else:
                    # Perform early stopping if validation loss doesn't improve
                    if epoch > 10:
                        not_increased += 1
                        # print('Not increased : {}/5'.format(not_increased))
                        if not_increased == early_stop:
                            print('Decrease LR')
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] / 2
                            not_increased = 0
                            end_train += 1
                        
                        if end_train == 2:
                            model.load_state_dict(best_model_wts)
                            y_true_val, y_pred_val = reshape_to_eval(xx_val,yy_val, model)
                            ev_val = explained_variance_score(y_true_val, y_pred_val)
                            return ev_val  # here change to not return a list but a single value for the trial to analyze
                        

        # Update learning rate with the scheduler
        scheduler.step()
        print("Epoch {:03} Train {:.4f} Val {:.4f}".format(epoch, train_losses[-1], val_losses[-1]))


        y_true_val_epoch, y_pred_val_epoch = reshape_to_eval(xx_val,yy_val, model)
        ev_val_epoch = explained_variance_score(y_true_val_epoch, y_pred_val_epoch)
        trial.report(ev_val_epoch, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    y_true_val, y_pred_val = reshape_to_eval(xx_val,yy_val, model)
    ev_val = explained_variance_score(y_true_val, y_pred_val)

    return ev_val


In [4]:
def reshape_to_eval(x,y, model):
    to_t_eval =  lambda array: torch.tensor(array, device='cpu', dtype=dtype)  
    x = to_t_eval(x) 
    y = to_t_eval(y)
    y_pred = model(x)
    y_array = y.detach().cpu().numpy()
    y_pred_array = y_pred.detach().cpu().numpy()

    # Reshape tensors to 2D arrays (flatten the batch and sequence dimensions)
    y_pred_2D = y_pred_array.reshape(-1, y_pred_array.shape[-1])
    y_true_2D = y_array.reshape(-1, y_array.shape[-1])
    
    return y_true_2D, y_pred_2D

In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(train_model_optuna, n_trials=100)

importance_scores = optuna.importance.get_param_importances(study)

# Print importance scores
for param, score in importance_scores.items():
    print(f"{param}: {score}")

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('Study statistics: ')
print("Number of finished trials: ", len(study.trials))
print("Number of pruned trials: ", len(pruned_trials))
print("Number of complete trials: ", len(complete_trials))

print('Best trial: ')
trial = study.best_trial

print("Loss: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-04-09 11:39:13,703] A new study created in memory with name: no-name-5c7a6a81-170f-421f-a823-918ee685923d


Epoch 000 Train 0.0198 Val 0.0145
Epoch 001 Train 0.0130 Val 0.0130
Epoch 002 Train 0.0115 Val 0.0124
Epoch 003 Train 0.0109 Val 0.0122
Epoch 004 Train 0.0104 Val 0.0118
Epoch 005 Train 0.0102 Val 0.0116
Epoch 006 Train 0.0100 Val 0.0117
Epoch 007 Train 0.0099 Val 0.0114
Epoch 008 Train 0.0098 Val 0.0114
Epoch 009 Train 0.0097 Val 0.0116
Epoch 010 Train 0.0097 Val 0.0114
Epoch 011 Train 0.0096 Val 0.0115
Epoch 012 Train 0.0096 Val 0.0117
Epoch 013 Train 0.0096 Val 0.0113
Epoch 014 Train 0.0096 Val 0.0113
Epoch 015 Train 0.0095 Val 0.0113
Epoch 016 Train 0.0095 Val 0.0113
Epoch 017 Train 0.0095 Val 0.0114
Epoch 018 Train 0.0096 Val 0.0115
Decrease LR
Epoch 019 Train 0.0095 Val 0.0114
Epoch 020 Train 0.0095 Val 0.0114
Epoch 021 Train 0.0095 Val 0.0113
Epoch 022 Train 0.0095 Val 0.0115
Epoch 023 Train 0.0095 Val 0.0114


[I 2024-04-09 11:41:08,719] Trial 0 finished with value: 0.6158506472905477 and parameters: {'n_hidden_units': 46, 'input_size_LSTM': 65, 'dropout_2': 0.10134575540103558, 'lr': 0.0005322351204250441, 'lr_gamma': 1.1002189401106162, 'seq_length_LSTM': 15, 'batch_size_train': 30, 'batch_size_val': 26, 'l1_ratio_reg': 0.5148661875809435}. Best is trial 0 with value: 0.6158506472905477.


Decrease LR
Epoch 000 Train 0.0154 Val 0.0137
Epoch 001 Train 0.0104 Val 0.0128
Epoch 002 Train 0.0093 Val 0.0122
Epoch 003 Train 0.0089 Val 0.0119
Epoch 004 Train 0.0086 Val 0.0118
Epoch 005 Train 0.0085 Val 0.0115
Epoch 006 Train 0.0084 Val 0.0115
Epoch 007 Train 0.0083 Val 0.0112
Epoch 008 Train 0.0083 Val 0.0113
Epoch 009 Train 0.0083 Val 0.0114
Epoch 010 Train 0.0083 Val 0.0113
Epoch 011 Train 0.0083 Val 0.0111
Epoch 012 Train 0.0083 Val 0.0115
Epoch 013 Train 0.0084 Val 0.0110
Epoch 014 Train 0.0084 Val 0.0107
Epoch 015 Train 0.0084 Val 0.0109
Epoch 016 Train 0.0084 Val 0.0110
Epoch 017 Train 0.0084 Val 0.0110
Epoch 018 Train 0.0084 Val 0.0108
Decrease LR
Epoch 019 Train 0.0084 Val 0.0108
Epoch 020 Train 0.0084 Val 0.0106
Epoch 021 Train 0.0084 Val 0.0107
Epoch 022 Train 0.0084 Val 0.0109
Epoch 023 Train 0.0084 Val 0.0107
Epoch 024 Train 0.0084 Val 0.0105
Epoch 025 Train 0.0084 Val 0.0108
Epoch 026 Train 0.0085 Val 0.0111
Epoch 027 Train 0.0085 Val 0.0106
Epoch 028 Train 0.0085 V

[I 2024-04-09 11:43:57,249] Trial 1 finished with value: 0.6399866739908854 and parameters: {'n_hidden_units': 45, 'input_size_LSTM': 56, 'dropout_2': 0.09380944879948282, 'lr': 0.0008950170631978813, 'lr_gamma': 1.0681999213972042, 'seq_length_LSTM': 14, 'batch_size_train': 25, 'batch_size_val': 29, 'l1_ratio_reg': 0.4775867506823128}. Best is trial 1 with value: 0.6399866739908854.


...

Epoch 001 Train 0.0126 Val 0.0121


[I 2024-04-09 12:54:01,041] Trial 98 pruned. 


Epoch 000 Train 0.0173 Val 0.0125
Epoch 000 Train 0.0144 Val 0.0116


[I 2024-04-09 12:54:15,597] Trial 99 pruned. 


Epoch 001 Train 0.0110 Val 0.0110
dropout_2: 0.28069458377073697
n_hidden_units: 0.178877168156399
lr_gamma: 0.16989039637889317
lr: 0.1228770099801176
batch_size_val: 0.09053367449017329
input_size_LSTM: 0.05426007696790645
batch_size_train: 0.040464044396067565
l1_ratio_reg: 0.035174142510194836
seq_length_LSTM: 0.02722890334951124
Study statistics: 
Number of finished trials:  100
Number of pruned trials:  76
Number of complete trials:  24
Best trial: 
Loss: 0.653988778591156
Best hyperparameters: {'n_hidden_units': 50, 'input_size_LSTM': 61, 'dropout_2': 0.11345183469124971, 'lr': 0.0012815987708498843, 'lr_gamma': 1.1790919403685756, 'seq_length_LSTM': 12, 'batch_size_train': 30, 'batch_size_val': 28, 'l1_ratio_reg': 0.5233302128551165}


### Plotting the study

In [6]:
optuna.visualization.plot_optimization_history(study)

In [7]:
#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

In [8]:
# Plotting the optimization history of the study.

optuna.visualization.plot_optimization_history(study)

#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

# Plotting the accuracy surface for the hyperparameters involved in the random forest model.

optuna.visualization.plot_contour(study, params=["seq_length_LSTM", "n_hidden_units"]) 